## Using python for data cleaning 

### first importing pandas for reading the excel file and for further data manipulation procedure

In [33]:
import pandas as pd 

### required data was present in two excel sheets so stored the excel sheets in two data frames 

In [ ]:
customer_df = pd.read_excel(r"File_path", sheet_name='Customer_Info')
customer_df.head()
account_df = pd.read_excel(r"File_path", sheet_name='Account_Info')
account_df.head()

,CustomerId,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,€0.0,1,Yes,2,Yes,1
1,15634602,€0.0,1,Yes,2,Yes,1
2,15647311,€83807.86,1,Yes,1,Yes,0
3,15619304,€159660.8,3,No,8,No,1
4,15701354,€0.0,2,No,1,No,0


### now we need to merge both data frame into a single df based on the common column "CustomerId"

In [56]:
df = pd.merge(customer_df,account_df,left_on='CustomerId', right_on='CustomerId', how='inner')
churn_df = df.copy()
churn_df.head()
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10004 entries, 0 to 10003
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10004 non-null  int64  
 1   Surname          10001 non-null  object 
 2   CreditScore      10004 non-null  int64  
 3   Geography        10004 non-null  object 
 4   Gender           10004 non-null  object 
 5   Age              10001 non-null  float64
 6   Tenure_x         10004 non-null  int64  
 7   EstimatedSalary  10004 non-null  object 
 8   Balance          10004 non-null  object 
 9   NumOfProducts    10004 non-null  int64  
 10  HasCrCard        10004 non-null  object 
 11  Tenure_y         10004 non-null  int64  
 12  IsActiveMember   10004 non-null  object 
 13  Exited           10004 non-null  int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 1.1+ MB


### there are two columnns with the same values Tenure x and Tenure y we need to drop one and keep a single column Tenure

In [57]:
churn_df.rename(columns={'Tenure_y': 'Tenure'}, inplace=True)
churn_df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure_x,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,Hargrave,619,FRA,Female,42.0,2,€101348.88,€0.0,1,Yes,2,Yes,1
1,15634602,Hargrave,619,FRA,Female,42.0,2,€101348.88,€0.0,1,Yes,2,Yes,1
2,15647311,Hill,608,Spain,Female,41.0,1,€112542.58,€83807.86,1,Yes,1,Yes,0
3,15619304,Onio,502,French,Female,42.0,8,€113931.57,€159660.8,3,No,8,No,1
4,15701354,Boni,699,FRA,Female,39.0,1,€93826.63,€0.0,2,No,1,No,0


In [58]:
churn_df.drop('Tenure_x', axis=1, inplace=True)
churn_df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,Hargrave,619,FRA,Female,42.0,€101348.88,€0.0,1,Yes,2,Yes,1
1,15634602,Hargrave,619,FRA,Female,42.0,€101348.88,€0.0,1,Yes,2,Yes,1
2,15647311,Hill,608,Spain,Female,41.0,€112542.58,€83807.86,1,Yes,1,Yes,0
3,15619304,Onio,502,French,Female,42.0,€113931.57,€159660.8,3,No,8,No,1
4,15701354,Boni,699,FRA,Female,39.0,€93826.63,€0.0,2,No,1,No,0


### Checking for the inconsistent values and correcting them

In [59]:
churn_df['Geography'].value_counts()
churn_df['Geography'] = churn_df['Geography'].replace({'French': 'France', 'FRA': 'France'})
churn_df['Geography'].value_counts()

Geography
France     5018
Germany    2509
Spain      2477
Name: count, dtype: int64

### now we need to look for the null values and either drop them or fill them

In [60]:
churn_df.isnull().sum()
churn_df.dropna(inplace=True)
churn_df.isnull().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
EstimatedSalary    0
Balance            0
NumOfProducts      0
HasCrCard          0
Tenure             0
IsActiveMember     0
Exited             0
dtype: int64

### here we will first check the the data types of each column and convert them to the right data type that we need for analysis

In [62]:
churn_df.dtypes
churn_df['EstimatedSalary'] = churn_df['EstimatedSalary'].replace('[^\d.]', '', regex=True).astype(float)
churn_df['Balance'] = churn_df['Balance'].replace('[^\d.]', '', regex=True).astype(float)
churn_df.dtypes

CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
EstimatedSalary    float64
Balance            float64
NumOfProducts        int64
HasCrCard           object
Tenure               int64
IsActiveMember      object
Exited               int64
dtype: object

### Now we need to map yes and no(categorical data) in 'HasCrCard' and 'IsActiveMember'column to numeric values that we will need later for aggregation 

In [63]:
churn_df['HasCrCard'].unique()
churn_df['HasCrCard'] = churn_df['HasCrCard'].map({'Yes': 1, 'No': 0})
churn_df['IsActiveMember'] = churn_df['IsActiveMember'].map({'Yes': 1, 'No': 0})
churn_df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,EstimatedSalary,Balance,NumOfProducts,HasCrCard,Tenure,IsActiveMember,Exited
0,15634602,Hargrave,619,France,Female,42.0,101348.88,0.00,1,1,2,1,1
1,15634602,Hargrave,619,France,Female,42.0,101348.88,0.00,1,1,2,1,1
2,15647311,Hill,608,Spain,Female,41.0,112542.58,83807.86,1,1,1,1,0
3,15619304,Onio,502,France,Female,42.0,113931.57,159660.80,3,0,8,0,1
4,15701354,Boni,699,France,Female,39.0,93826.63,0.00,2,0,1,0,0


### Renaming the column for better readablity and use

In [64]:
print(churn_df.columns.tolist())
churn_df = churn_df.rename(columns={
    'CustomerId': 'customer_id',
    'Surname': 'surname',
    'CreditScore': 'credit_score',
    'Geography': 'geography',
    'Gender': 'gender',
    'Age': 'age',
    'EstimatedSalary': 'estimated_salary',
    'Balance': 'balance',
    'NumOfProducts': 'num_of_products',
    'HasCrCard': 'has_cr_card',
    'Tenure': 'tenure',
    'IsActiveMember': 'is_active_member',
    'Exited': 'churned'
})

['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'EstimatedSalary', 'Balance', 'NumOfProducts', 'HasCrCard', 'Tenure', 'IsActiveMember', 'Exited']


### checking for duplicate rows and droping them 

In [65]:
print("Number of duplicate rows = ", churn_df.duplicated().sum())

Number of duplicate rows =  4


In [66]:
churn_df = churn_df.drop_duplicates()
print("Number of duplicate rows = ", churn_df.duplicated().sum())

Number of duplicate rows =  0


### now checking for outliers or any inconsistencies in columns 

In [67]:
churn_df.describe()

,customer_id,credit_score,age,estimated_salary,balance,num_of_products,has_cr_card,tenure,is_active_member,churned
count,9.997000e+03,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000
mean,1.569094e+07,650.545364,38.922077,100092.222656,76482.679807,1.530359,0.515055,5.013204,0.515055,0.203761
std,7.193443e+04,96.657932,10.489072,57518.775702,62397.174721,0.581669,0.499798,2.892364,0.499798,0.402814
min,1.556570e+07,350.000000,18.000000,11.580000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.562853e+07,584.000000,32.000000,50974.570000,0.000000,1.000000,0.000000,3.000000,0.000000,0.000000
50%,1.569073e+07,652.000000,37.000000,100236.020000,97188.620000,1.000000,1.000000,5.000000,1.000000,0.000000
75%,1.575323e+07,718.000000,44.000000,149399.700000,127642.440000,2.000000,1.000000,7.000000,1.000000,0.000000
max,1.581569e+07,850.000000,92.000000,199992.480000,250898.090000,4.000000,1.000000,10.000000,1.000000,1.000000


### Again for the last time verifing the data types and for null values

In [68]:
churn_df.dtypes
churn_df.isnull().sum()

customer_id         0
surname             0
credit_score        0
geography           0
gender              0
age                 0
estimated_salary    0
balance             0
num_of_products     0
has_cr_card         0
tenure              0
is_active_member    0
churned             0
dtype: int64

### exporting the cleaned dataset to csv file 

In [72]:
churn_df.to_csv("churn_cleaned.csv", index=False)